<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h1 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<span dir="ltr">M1.8</span> — استخدام ملاحظات ناسا للأرض
</h1>

<hr />

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<em>جزء من:</em>
<a dir="ltr" href="https://github.com/OpenClimateScience/M1-Open-Climate-Data"><strong>Open Climate Data</strong></a>
<span dir="ltr"> | </span>
<strong>الدرس السابق</strong>
<span dir="ltr"> | </span>
<strong>الدرس التالي</strong>
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>المحتويات:</strong>
</p>

<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
  <li><a dir="ltr" href="#Organizing-our-file-system">تنظيم نظام الملفات</a></li>
  <li>
    <a dir="ltr" href="#Using-NASA-climate-observations">استخدام ملاحظات ناسا المناخية</a>
    <ul>
      <li><a dir="ltr" href="#Downloading-SMAP-Level-3-soil-moisture-data">تنزيل بيانات رطوبة التربة <span dir="ltr">SMAP</span> المستوى <span dir="ltr">3</span></a></li>
      <li><a dir="ltr" href="#Customizing-an-Earthdata-Search-download">تخصيص التنزيل عبر <span dir="ltr">Earthdata Search</span></a></li>
    </ul>
  </li>
  <li>
    <a dir="ltr" href="#Understanding-hierarchical-data-files-(HDF5)">فهم ملفات البيانات الهرمية (<span dir="ltr">HDF5</span>)</a>
    <ul>
      <li><a dir="ltr" href="#Reading-HDF5-datasets">قراءة مجموعات بيانات <span dir="ltr">HDF5</span></a></li>
      <li><a dir="ltr" href="#Subsetting-the-SMAP-L3-data">اقتطاع بيانات <span dir="ltr">SMAP L3</span></a></li>
    </ul>
  </li>
  <li><a dir="ltr" href="#Masking-out-permanent-water-bodies">استبعاد المسطحات المائية الدائمة</a></li>
  <li><a dir="ltr" href="#Using-quality-assurance-bit-flags">استخدام أعلام ضمان الجودة (<span dir="ltr">QA</span>) على مستوى البِتّ</a></li>
  <li>
    <a dir="ltr" href="#Creating-a-soil-moisture-time-series">إنشاء سلسلة زمنية لرطوبة التربة</a>
    <ul>
      <li><a dir="ltr" href="#Calculating-a-moving-average">حساب متوسط متحرك</a></li>
    </ul>
  </li>
</ul>

<hr />
<hr />

</div>


In [ ]:
import earthaccess
import numpy as np
import xarray as xr
from matplotlib import pyplot

auth = earthaccess.login()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تنظيم نظام الملفات
</h2>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
مرة أخرى، سنحتاج إلى مكان لتخزين البيانات الخام. عندما بدأنا العمل مع بيانات
<span dir="ltr">Noah NLDAS</span>
أنشأنا المجلدات التالية في نظام الملفات لدينا:
</p>

<pre><code dir="ltr">data_raw/
  NLDAS
  SMAP_L3</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تأكد من وجود مجلد
<span dir="ltr"><code>SMAP_L3</code></span>
أيضًا ليستقبل البيانات التي سنقوم بتنزيلها الآن.
</p>

<hr />

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
استخدام ملاحظات ناسا المناخية
</h2>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تُعد بيانات
<span dir="ltr">NLDAS</span>
أداة ممتازة للدراسات الاسترجاعية، لكن باعتبارها بيانات إعادة تحليل
(<span dir="ltr">re-analysis</span>)،
فلها بعض القيود:
</p>

<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
  <li>زمن تأخر مرتفع نسبيًا في إتاحة البيانات.</li>
  <li>دمج مصادر متعددة بمستويات دقة وتغطية مكانية متفاوتة.</li>
</ul>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
إذا أردنا توصيف الجفاف الومضي أو رصده شبه لحظيًا، فلا ينبغي الاعتماد على بيانات إعادة التحليل.
بدلًا من ذلك، نحتاج إلى ملاحظات مباشرة لظروف الجفاف.
<strong>دعونا نرى ما الذي يمكن أن نتعلمه عن جفاف الومضة لعام <span dir="ltr">2017</span> من تقديرات ناسا لرطوبة التربة المعتمدة على الأقمار الصناعية.</strong>
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>سنستخدم بيانات مهمة ناسا لرطوبة التربة النشطة/المنفعلة (<span dir="ltr">SMAP</span>).</strong>
توفر مهمات رصد الأرض في ناسا بيانات مُصنَّفة إلى مستويات معالجة مختلفة:
</p>

<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
  <li><strong>المستوى <span dir="ltr">1</span>:</strong> بيانات خام مقاسة مباشرة.</li>
  <li><strong>المستوى <span dir="ltr">2</span>:</strong> بيانات مشتقة قابلة للتفسير الفيزيائي بنفس الدقة الأصلية.</li>
  <li><strong>المستوى <span dir="ltr">3</span>:</strong> بيانات موحّدة على شبكة مكانية وسلسلة زمنية منتظمة.</li>
  <li><strong>المستوى <span dir="ltr">4</span>:</strong> بيانات مُعزَّزة بالنمذجة ودمج مصادر إضافية.</li>
</ul>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تنزيل بيانات رطوبة التربة <span dir="ltr">SMAP</span> المستوى <span dir="ltr">3</span>
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
سنستخدم بيانات رطوبة التربة السطحية (بدقة <span dir="ltr">36</span> كم) من المستوى <span dir="ltr">3</span> لمهمة
<span dir="ltr">SMAP</span>
لأنها تمثل توازنًا جيدًا بين الملاحظة المباشرة وسهولة الاستخدام.
</p>

<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
  <li>سنستخدم <span dir="ltr">Earthdata Search</span> للوصول إلى البيانات.</li>
  <li>هذه البيانات ليست متاحة مباشرة في السحابة، لذا سنقوم بتنزيلها.</li>
  <li>سنضع البيانات داخل مجلد <span dir="ltr"><code>data_raw/SMAP_L3</code></span>.</li>
</ul>

<hr />

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
🎯 أفضل الممارسات
</h4>

<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
  <li>
    داخل شجرة ملفات <span dir="ltr">Jupyter Notebook</span>، وفي مجلد
    <span dir="ltr"><code>SMAP_L3</code></span>،
    أنشئ ملفًا جديدًا باسم
    <span dir="ltr"><code>README.txt</code></span>.
  </li>
  <li>
    افتح الملف وأضف معلومات مفيدة حول مصدر البيانات وخطوات تنزيلها.
    فيما يلي مثال توضيحي.
  </li>
</ul>

<pre><code dir="ltr">Author: Jane Q. Public (jane.public@example.com)
Date: November 1, 2023

This folder contains Level 3 data from the SMAP Mission. It was downloaded from:

    https://search.earthdata.nasa.gov/search?q=SPL3SMP+V008

Here's some more information about this product:

    https://nsidc.org/data/spl3smp/versions/8</code></pre>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
قد يبدو هذا قدرًا بسيطًا من التوثيق، لكنه يتضمن معلومات أساسية قد نحتاجها لاحقًا.
<strong>أي توثيق—even القليل—أفضل من عدم التوثيق.</strong>
</p>

<hr />

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تخصيص تنزيل عبر <span dir="ltr">Earthdata Search</span>
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يمرّ قمر
<span dir="ltr">SMAP</span>
فوق كل موقع مرتين يوميًا: مرور «صباحي» ومرور «بعد الظهر» (بالتوقيت المحلي).
سنستخدم بيانات رطوبة التربة من مرور بعد الظهر
(<span dir="ltr">PM</span>)،
لأن هذا هو الوقت الذي يكون فيه إجهاد رطوبة التربة على الغطاء النباتي في ذروته على الأرجح.
</p>

<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
  <li>
    <a dir="ltr" href="https://search.earthdata.nasa.gov/search?q=SPL3SMP%20V008"><strong>سينقلك هذا الرابط إلى نقطة البداية الصحيحة.</strong></a>
    انقر على مجموعة البيانات الوحيدة المعروضة في الجهة اليمنى من نافذة البحث.
  </li>
  <li>
    سنقوم بتنزيل بيانات شهري أغسطس وسبتمبر لدراسة بداية وتطوّر جفاف الومضة لعام
    <span dir="ltr">2017</span>:
    <strong>اختر نطاقًا زمنيًا من
    <span dir="ltr">2017-06-01</span>
    إلى
    <span dir="ltr">2017-09-30</span>.</strong>
  </li>
  <li>
    في الزاوية السفلية اليمنى،
    <strong>انقر الزر الأخضر الكبير الذي يحمل عبارة
    <span dir="ltr">"Download All"</span>.</strong>
  </li>
  <li>
    حجم
    <span dir="ltr">3.8</span>
    غيغابايت كبير!
    هل يمكننا تقليل حجم التنزيل؟
    نحن مهتمون فقط برطوبة التربة من مرور بعد الظهر.
    <strong>انقر
    <span dir="ltr">"Edit Options"</span>
    ثم، تحت
    <span dir="ltr">"Select a data access method"</span>،
    اختر خيار
    <span dir="ltr">"Customize"</span>.</strong>
  </li>
</ul>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<img src="assets/M1_Earthdata_Search_SMAP-L3_customize_order.png" alt="" />
</p>

<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
  <li>
    <strong>مرّر للأسفل إلى
    <span dir="ltr">"Configure data customization options"</span>
    ثم إلى
    <span dir="ltr">"Band subsetting"</span>.</strong>
  </li>
  <li>
    <strong>داخل مربع النص الذي يحمل عبارة
    <span dir="ltr">"Filter"</span>
    اكتب
    <span dir="ltr"><code>soil_moisture_dca_pm</code></span>.</strong>
    سيؤدي ذلك إلى تصفية المتغيرات المتاحة («الحزم»)
    لإظهار هذا المتغير فقط، وهو تقدير رطوبة التربة باستخدام خوارزمية القناتين
    (<span dir="ltr">DCA</span>)
    لمرور بعد الظهر
    (<span dir="ltr">PM</span>).
  </li>
  <li>
    للتأكد من أننا ننزّل الحقول التي نحتاجها فقط،
    <strong>ستحتاج إلى إلغاء تحديد المربع بجانب
    <span dir="ltr"><code>SPL3SMP</code></span>
    ثم إعادة تحديد المربع بجانب
    <span dir="ltr"><code>soil_moisture_dca_pm</code></span>
    (انظر لقطة الشاشة أدناه)،
    إضافةً إلى كل متغير نرغب في الاحتفاظ به.</strong>
  </li>
</ul>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<img src="assets/M1_Earthdata_Search_SMAP-L3_customize_order_variables.png" alt="" />
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>نريد تنزيل الحقول التالية فقط:</strong>
</p>

<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
  <li><span dir="ltr"><code>soil_moisture_dca_pm</code></span></li>
  <li><span dir="ltr"><code>static_water_body_fraction_pm</code></span></li>
  <li><span dir="ltr"><code>retrieval_qual_flag_dca_pm</code></span></li>
</ul>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
اضغط
<span dir="ltr">"Done"</span>
أسفل هذا النموذج،
ثم الزر الأخضر الكبير الذي يحمل عبارة
<span dir="ltr">"Download Data"</span>.
</p>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

#### 🚩 <span style="color:red">لكن انتظر!</span>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
نظرًا لأننا اخترنا مجموعة فرعية فقط من المتغيّرات، فسيتعيّن علينا الانتظار إلى أن تصلنا رسالة بريد إلكتروني تُفيد بأن طلب التنزيل أصبح جاهزًا.
<strong>لا تحتاج إلى تنفيذ هذه الخطوات بنفسك، لأنني قمتُ مسبقًا بإعداد جميع الحُبيبات (granules) التي كان سيتم تنزيلها بهذه الطريقة.</strong>
يمكن تنزيلها مباشرة من هنا:
</p>

<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
  <li>
    <a dir="ltr" href="http://files.ntsg.umt.edu/data/ScienceCore/SMAP_L3_SPL3SMP_V008_20170601_20170930.zip">
      SMAP_L3_SPL3SMP_V008_20170601_20170930.zip
    </a>
    (قم باستخراج محتويات ملف
    <span dir="ltr">ZIP</span>
    هذا إلى المجلد
    <span dir="ltr"><code>data_raw/SMAP_L3</code></span>)
  </li>
</ul>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<hr />

<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
فهم ملفات البيانات الهرمية (<span dir="ltr">HDF5</span>)
</h2>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يتم تخزين كل ملف من بيانات
<span dir="ltr">SMAP</span>
المستوى
<span dir="ltr">3</span>
التي قمنا بتنزيلها على هيئة
<strong>ملف بيانات هرمي من الإصدار
<span dir="ltr">5</span>
(<span dir="ltr">HDF5</span>)</strong>.
</p>

</div>


In [ ]:
import h5py

filename = 'data_raw/SMAP_L3/SMAP_L3_SM_P_20170901_R18290_001.h5'
hdf = h5py.File(filename, 'r')
hdf

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يشبه ملف
<span dir="ltr">HDF5</span>
إلى حدٍّ كبير ملف
<span dir="ltr">netCDF4</span>:
كلاهما تنسيقان هرميان قادران على تخزين مجموعات بيانات متعددة وبيانات وصفية (metadata)
داخل ملف واحد.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
ماذا نعني بكونه «هرميًا»؟
يمكنك التفكير في ملف
<span dir="ltr">HDF5</span>
أو
<span dir="ltr">netCDF4</span>
على أنه يشبه شجرة ملفات:
حيث يمكن تنظيم
<em>مجموعات البيانات</em>
داخل
<em>مجموعات</em>
(<span dir="ltr">groups</span>)
متداخلة.
كما يمكن إرفاق البيانات الوصفية، على شكل
<em>سمات</em>
(<span dir="ltr">attributes</span>)،
بأي مجموعة أو مجموعة بيانات داخل الملف.
</p>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<img src="assets/hdf5-structure.jpg" alt="" />
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<em>الصورة بإذن من
<span dir="ltr">NEON Science</span>.</em>
</p>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يمكننا الاطلاع على المجموعات ومجموعات البيانات الموجودة في أعلى مستوى من هذا التسلسل الهرمي بكتابة:
</p>

</div>


In [ ]:
hdf.keys()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يُتعامل مع كائن
<span dir="ltr"><code>h5py.File</code></span>
المسمّى
<span dir="ltr"><code>hdf</code></span>
كما لو كان قاموس بايثون. فإذا أردنا النظر إلى مجموعة
<span dir="ltr"><code>'Metadata'</code></span>
مثلًا، نكتب:
</p>

</div>


In [ ]:
hdf['Metadata']

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
هذا ليس مُفيدًا كثيرًا، لكن كل مجموعة وكل مجموعة بيانات داخل كائن
<span dir="ltr"><code>h5py.File</code></span>
تتصرّف أيضًا مثل قاموس بايثون:
</p>

</div>


In [ ]:
hdf['Metadata'].keys()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تُعد مجموعة
<span dir="ltr"><code>'Metadata'</code></span>
مثالًا على كيفية تخزين معلومات داخل ملف
<span dir="ltr">HDF5</span>
غير المصفوفات متعددة الأبعاد.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
ما أهمية هذه المجموعة الفارغة؟
</p>

</div>


In [ ]:
hdf['Metadata/ProcessStep']

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تمامًا كما في ملفات
<span dir="ltr">netCDF</span>،
يمكن وسم كل مجموعة بيانات في ملف
<span dir="ltr">HDF5</span>
بسمات (attributes).
</p>

</div>


In [ ]:
hdf['Metadata/ProcessStep'].attrs.keys()

In [ ]:
hdf['Metadata/ProcessStep'].attrs['processor']

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
قراءة مجموعات بيانات <span dir="ltr">HDF5</span>
</h3>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
نفتح ملف
<span dir="ltr">HDF5</span>
للقراءة باستخدام العلامة
<span dir="ltr"><code>'r'</code></span>
كما يلي.
</p>

</div>


In [ ]:
hdf = h5py.File(filename, 'r')
hdf.keys()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
مرة أخرى، يمكننا الوصول إلى مجموعات البيانات بشكل هرمي...
</p>

</div>


In [ ]:
hdf['Soil_Moisture_Retrieval_Data_PM'].keys()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
وإذا أردنا قراءة مجموعة بيانات، نستخدم ترميز
<span dir="ltr"><code>[:]</code></span>
في
<span dir="ltr">NumPy</span>
للدلالة على أننا نريد الوصول إلى مصفوفة.
</p>

</div>


In [ ]:
hdf['Soil_Moisture_Retrieval_Data_PM/soil_moisture_dca_pm'][:]

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
عندما ننتهي من العمل مع ملف
<span dir="ltr">HDF5</span>
مفتوح، ينبغي التأكد من إغلاقه.
</p>

</div>


In [ ]:
hdf.close()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>دعونا نرى ما المختلف عند فتح الملف نفسه باستخدام
<span dir="ltr"><code>xarray</code></span>.</strong>
وبالتحديد، انظر إلى
<strong>متغيرات البيانات</strong>.
</p>

</div>


In [ ]:
ds = xr.open_dataset(filename)
ds

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
المتغير الوحيد الذي تم العثور عليه،
<span dir="ltr"><code>"crs"</code></span>،
لن يكون مفيدًا لنا كثيرًا.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>لدى
<span dir="ltr"><code>xarray</code></span>
قيود عند فتح ملفات
<span dir="ltr">HDF5</span>؛ إذ لا يستطيع تحديد المجموعات المتاحة.</strong>
لذلك، علينا أن نحدد المجموعة التي نريد فتحها.
</p>

</div>


In [ ]:
ds = xr.open_dataset(filename, group = 'Soil_Moisture_Retrieval_Data_PM')
ds

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
الآن لدينا متغير مفيد،
<span dir="ltr"><code>"soil_moisture_dca_pm"</code></span>،
لكن
<span dir="ltr"><code>xarray.Dataset</code></span>
الخاص بنا لا يحتوي على إحداثيات!
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
إحدى طرق إصلاح ذلك هي تعيين إحداثيات إلى
<span dir="ltr"><code>xarray.Dataset</code></span>.
ولهذا نحتاج إلى مكتبة
<span dir="ltr"><code>h5py</code></span>
المتخصصة في التعامل مع ملفات
<span dir="ltr">HDF5</span>.
يمكننا قراءة إحداثيات
<span dir="ltr"><code>"x"</code></span>
و
<span dir="ltr"><code>"y"</code></span>
من ملف
<span dir="ltr"><code>h5py.File</code></span>
ثم كتابتها إلى
<span dir="ltr"><code>xarray.Dataset</code></span>
كما يلي.
</p>

</div>


In [ ]:
# Instructor: Note coordinates assignment

hdf = h5py.File(filename, 'r')
ds = ds.assign_coords({'x': hdf['x'][:], 'y': hdf['y'][:]})
hdf.close()
ds

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
الآن بعد أن أصبح لدينا <strong>متغير بيانات</strong> و<strong>إحداثيات</strong>، أصبحنا جاهزين لرسم البيانات!
</p>

</div>


In [ ]:
pyplot.figure(figsize = (12, 5))
ds['soil_moisture_dca_pm'].plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

#### 🚩 <span style="color:red">انتبه</span>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>لاحظ التخطيط المخطط (striping) في هذه الصورة.</strong>
يمتلك قمر
<span dir="ltr">SMAP</span>
زمن إعادة زيارة يتراوح بين يومين وثلاثة أيام.
هذا يعني أنه في يوم واحد، يقوم راديومتر القمر بتصوير جزء فقط من الكرة الأرضية.
يمكننا دمج مرور الصباح
(<span dir="ltr"><code>"soil_moisture_dca_am"</code></span>)
ومرور بعد الظهر
(<span dir="ltr"><code>"soil_moisture_dca_pm"</code></span>)
لليوم نفسه، لكن رطوبة التربة في كثير من مناطق العالم تتغير كثيرًا بين الصباح وبعد الظهر،
ولذلك قد لا يكون هذا معقولًا.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
اخترنا مرور بعد الظهر
(<span dir="ltr"><code>"soil_moisture_dca_pm"</code></span>)
لأن بعد الظهر غالبًا ما يكون الوقت الذي يكون فيه إجهاد رطوبة التربة أعلى في النظم البيئية البرّية.
</p>

</div>


In [ ]:
def process_smap_l3(file_path):
    '''
    Parameters
    ----------
    file_path : str
        The file path to the SMAP L3 file

    Returns
    -------
    xarray.Dataset
    '''
    with h5py.File(file_path, 'r') as hdf:
        ds = xr.open_dataset(file_path, group = 'Soil_Moisture_Retrieval_Data_PM')
        return ds.assign_coords({'x': hdf['x'][:], 'y': hdf['y'][:]})

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

---

### اقتطاع بيانات <span dir="ltr">SMAP L3</span>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بيانات رطوبة التربة من
<span dir="ltr">SMAP</span>
عالمية، لكننا حاليًا مهتمون بمنطقة دراسة صغيرة: السهول الشمالية في الولايات المتحدة.
كيف يمكننا اقتطاع بيانات
<span dir="ltr">SMAP</span>
لتغطية منطقة اهتمامنا فقط؟
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
ربما لاحظت أن الإحداثيات التي أضفناها إلى
<span dir="ltr"><code>xarray.Dataset</code></span>
أعلاه ليست إحداثيات خط عرض/خط طول.
تُسقَط بيانات
<span dir="ltr">SMAP</span>
على إسقاط
<span dir="ltr">EASE-Grid 2.0</span>،
حيث ترمز
<span dir="ltr">EASE</span>
إلى
<span dir="ltr">Equal-Area Scalable Earth</span>.
هذا الإسقاط العالمي له مزايا عديدة، لكن إحداثيات
<span dir="ltr">X</span>
(<span dir="ltr">Easting</span>)
و
<span dir="ltr">Y</span>
(<span dir="ltr">Northing</span>)
قد تكون صعبة الفهم عندما نكون معتادين على إحداثيات خط العرض/خط الطول.
</p>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>
فيما يلي مثال لكيفية تعديل دالة
<span dir="ltr"><code>process_smap_l3()</code></span>
لتحويل إحداثيات مجموعة البيانات من
<span dir="ltr">EASE-Grid 2.0</span>
إلى إحداثيات خط الطول وخط العرض باستخدام مكتبة
<span dir="ltr"><code>pyproj</code></span>.
</strong>
</p>

</div>

```python
from pyproj import Transformer
from pyproj import CRS

def process_smap_l3(file_path):
    '''
    Parameters
    ----------
    file_path : str
        The file path to the SMAP L3 file

    Returns
    -------
    xarray.Dataset
    '''
    # Get a function to convert coordinates from EASE-Grid 2.0 to WGS84 (latitude-longitude)
    crs_ease2 = CRS.from_epsg(6933)
    crs_wgs84 = CRS.from_epsg(4326)
    transform = Transformer.from_crs(crs_ease2, crs_wgs84)
    
    with h5py.File(file_path, 'r') as hdf:
        # Convert from (Northing, Easting) to (Latitude, Longitude)
        lng = list(map(lambda x: transform.transform(x, 0)[1], hdf['x'][:].tolist()))
        lat = list(map(lambda y: transform.transform(0, y)[0], hdf['y'][:].tolist()))
        ds = xr.open_dataset(file_path, group = 'Soil_Moisture_Retrieval_Data_PM')
        
    return ds.assign_coords({'x': lng, 'y': lat})
```
<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
👉 <strong>للحفاظ على بساطة الأمور، لن نقوم بإجراء تحويل الإحداثيات في درس اليوم.</strong>
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بدلًا من ذلك، سنقوم باقتطاع بياناتنا باستخدام نطاق من الصفوف والأعمدة
يقابل منطقة قريبة من مدينة
<span dir="ltr">Glasgow</span>،
ولاية
<span dir="ltr">Montana</span>:
خط طول
<span dir="ltr">105.882° W</span>
وخط عرض
<span dir="ltr">48.059° N</span>.
</p>

</div>


In [ ]:
ds = process_smap_l3('data_raw/SMAP_L3/SMAP_L3_SM_P_20170802_R18290_001.h5')

ds['soil_moisture_dca_pm'][49:59,187:227].plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

#### 🚩 <span style="color:red">انتبه</span>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
هناك أمران ينبغي ملاحظتهما من هذه الصورة:
</p>

<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
  <li>
    <strong>من الواضح أن الجزء الغربي من منطقة دراستنا لم يلتقطه القمر في هذا اليوم.</strong>
    وهذا يعني أنه، اعتمادًا على اليوم، قد تعكس القيمة ظروف رطوبة التربة في أجزاء مختلفة (وأصغر) من منطقة الدراسة.
  </li>
  <li>
    <strong>توجد منطقة ذات رطوبة تربة عالية جدًا (بكسل أصفر ساطع) في يسار-وسط الصورة.</strong>
    هذا البكسل على الأرجح مسطح مائي دائم، لذا إذا كنا نجري تحليلًا إقليميًا فسنرغب في استبعاده من التحليل.
  </li>
</ul>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
دعونا نعالج هاتين المشكلتين.
</p>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

---

## استبعاد المسطحات المائية الدائمة

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
دعونا نلقي نظرة على مجموعة البيانات
<span dir="ltr"><code>static_water_body_fraction_pm</code></span>.
</p>

</div>


In [ ]:
pyplot.figure(figsize = (12, 5))
ds['static_water_body_fraction_pm'].plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
على الرغم من أن نسبة المسطح المائي «ثابتة» ولا تتغير مع الزمن،
إلا أن مجموعة بيانات
<span dir="ltr">SMAP L3</span>
تعرض فقط ذلك الجزء من القناع الذي تم فيه الحصول على بيانات،
ولهذا يظهر التخطيط المخطط (striping).
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
ولنركّز على منطقة دراستنا.
</p>

</div>


In [ ]:
ds['static_water_body_fraction_pm'][49:59,187:227].plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
كيف يمكننا استخدام ذلك لاستبعاد المسطحات المائية الدائمة من بيانات رطوبة التربة لدينا؟
سنحتاج إلى تحديد عتبة (threshold) لنسبة تغطية الماء تُستخدم لتعريف «مسطحات مائية دائمة».
لنعتمد الاصطلاح التالي: البكسلات التي تزيد نسبة المساحة المغطاة بالماء فيها عن
<span dir="ltr">20%</span>
(أي أن مساحة الماء الكسرية
<span dir="ltr">$\ge 0.2$</span>)
ينبغي استبعادها (mask).
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تُذكّرنا الصورة أعلاه بمشكلة أخرى نحتاج إلى التعامل معها:
كل صورة يومية قد تتضمن جزءًا فقط من منطقة دراستنا.
<strong>
في كل حبيبة (granule) يومية، يمكننا استخدام مجموعة البيانات
<span dir="ltr"><code>static_water_body_fraction_pm</code></span>
لإنشاء قناع ثنائي بحيث تشير
<span dir="ltr">1</span>
إلى مسطح مائي دائم وتشير
<span dir="ltr">0</span>
إلى أي شيء آخر (بيانات صالحة).
</strong>
</p>

</div>


In [ ]:
np.where(ds['static_water_body_fraction_pm'][:] >= 0.2, 1, 0)[49:59,187:227]

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
دعونا نُحدّث دالة
<span dir="ltr"><code>process_smap_l3()</code></span>
لتشمل استبعاد (masking) البيانات.
<strong>
لاحظ أننا أضفنا وسيطًا بكلمة مفتاحية (keyword argument) إلى دالتنا لمنح المستخدمين تحكمًا في عتبة مساحة الماء الكسرية المستخدمة.
ومن خلال توفير قيمة افتراضية للوسيط
<span dir="ltr"><code>threshold</code></span>،
فنحن نوثق ضمنيًا قرارنا باستخدام
<span dir="ltr"><code>0.2</code></span>
كعتبة.
</strong>
</p>

</div>


In [ ]:
def process_smap_l3(file_path, threshold = 0.2):
    '''
    Parameters
    ----------
    file_path : str
        The file path to the SMAP L3 file
    threshold : float
        The fractional water area threshold to use when masking out
        permnanet water bodies

    Returns
    -------
    xarray.Dataset
    '''
    with h5py.File(file_path, 'r') as hdf:
        ds = xr.open_dataset(file_path, group = 'Soil_Moisture_Retrieval_Data_PM')
        ds = ds.assign_coords({'x': hdf['x'][:], 'y': hdf['y'][:]})

    # Create a binary (0 or 1) array based on the water fraction threshold
    mask = np.where(ds['static_water_body_fraction_pm'][:] >= threshold, 1, 0)
    
    # Read the dataset out as a NumPy array so we can mask it
    data = ds['soil_moisture_dca_pm'].to_numpy()
    
    # Write NaN into this dataset wherever there are permanent water bodies
    data[mask == 1] = np.nan
    ds['soil_moisture_dca_pm'][:] = data
    return ds

In [ ]:
ds = process_smap_l3('data_raw/SMAP_L3/SMAP_L3_SM_P_20170802_R18290_001.h5')
ds['soil_moisture_dca_pm'][49:59,187:227].plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<hr />

<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
استخدام أعلام ضمان الجودة على مستوى البِتّ
</h2>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
اعتبار آخر عند استخدام مجموعات بيانات رصد الأرض المعتمدة على الأقمار الصناعية هو
<strong>معلومات ضمان الجودة (<span dir="ltr">QA</span>)</strong>.
أحيانًا يفشل مستشعر القمر في استرجاع قيمة جيدة لبكسل معيّن،
ربما بسبب ظروف غائمة (في حالة المستشعرات البصرية)
أو بسبب خلل مؤقت في الأداة أو البرمجيات.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
عادةً ما تُخزَّن هذه المعلومات كحزمة (band) منفصلة ضمن بيانات راستر متعددة الحزم،
أو—كما في حالتنا—كمجموعة بيانات منفصلة داخل ملف هرمي.
يمتلك ملف
<span dir="ltr">QA</span>
قيمة لكل بكسل في الصورة، وتحمل تلك القيمة أنواعًا مختلفة من المعلومات حول جودة الاسترجاع.
</p>

</div>


In [ ]:
flags = ds['retrieval_qual_flag_dca_pm'].to_numpy()

# Get each unique value in this dataset
np.unique(flags)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>ماذا تعني هذه الأرقام؟</strong>
كل رقم عشري أعلاه هو في الواقع تمثيل <em>مضغوط</em> (packed) لعدة أعلام
<span dir="ltr">QA</span>.
كل علم
<span dir="ltr">QA</span>
هو
<span dir="ltr">1</span>
أو
<span dir="ltr">0</span>
وفق تمثيل ثنائي.
إليك مثالًا على ذلك من مهمة ناسا
<span dir="ltr">MODIS</span>:
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<img src="./assets/bit-packing.png" alt="" />
</p>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يوضح المثال أعلاه التمثيل الثنائي للعدد
<span dir="ltr">64</span>.
نعرف أن القيمة العشرية المقابلة هي
<span dir="ltr">64</span>
لأن هناك قيمة
<span dir="ltr">1</span>
واحدة فقط، وهي في الموضع <em>السادس</em> عند العد من اليمين.
وباستخدام الأساس
<span dir="ltr">2</span>
(لأن الأرقام الثنائية اثنان) ورفعِه للقوة السادسة، نحصل على
<span dir="ltr">64</span>.
</p>

</div>


In [ ]:
2**6

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
وبصيغة أخرى، هكذا نحول العدد الثنائي
<span dir="ltr"><code>01000000</code></span>
(من الأعلى) إلى عدد عشري:
</p>

<div dir="ltr" style="unicode-bidi: isolate; text-align: left;">
$$
64 = (0\times 2^7) + (1\times 2^6) + (0\times 2^5) + (0\times 2^4) + (0\times 2^3) + (0\times 2^2) + (0\times 2^1) + (0\times 2^0)
$$
</div>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
كل رقم ثنائي هو <em>بِتّ</em> (<span dir="ltr">bit</span>).
في المثال أعلاه، تمثل مجموعات من بِتّ واحد أو أكثر (bitwords) فئات مختلفة من معلومات
<span dir="ltr">QA</span>.
في مثال
<span dir="ltr">MODIS</span>
هذا، بالنسبة للكلمة-البتية رقم
<span dir="ltr">2</span>
(<span dir="ltr">"Bit No.2"</span>)،
تشير
<span dir="ltr">0</span>
إلى أن «الكواشف تبدو سليمة»، بينما تشير
<span dir="ltr">1</span>
إلى خلاف ذلك.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>كيف يمكننا استخدام هذه المعلومات؟</strong>
أولًا، ينبغي الرجوع إلى
<a dir="ltr" href="https://nsidc.org/sites/default/files/spl3smp-v008-userguide.pdf">دليل مستخدم <span dir="ltr">SMAP L3</span></a>
والنظر إلى الجدول
<span dir="ltr">6</span>.
يمكن العثور على دليل المستخدم من
<a dir="ltr" href="https://nsidc.org/data/spl3smp/versions/8">صفحة منتج <span dir="ltr">SMAP L3</span> على موقع <span dir="ltr">NSIDC</span></a>.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يوضح دليل المستخدم معنى كل كلمة-بتية.
وبالنسبة لحالة استخدامنا، نريد استخدام بيانات
<span dir="ltr">SMAP L3</span>
التي تكون فيها «عملية استرجاع رطوبة التربة ذات جودة موصى بها»،
ووفق الجدول
<span dir="ltr">6</span>
فهذا يعني أن البِتّ الأيمن (الأخير) يجب أن يساوي
<span dir="ltr">0</span>.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>
كيف نعرف أي القيم العشرية في مجموعة البيانات
<span dir="ltr"><code>retrieval_qual_flag_dca_pm</code></span>
توافق هذه الحالة (حيث يكون البِتّ الأيمن
<span dir="ltr">0</span>)؟
</strong>
نلاحظ من التحويل الثنائي↔العشري أن كل عدد عشري <em>فردي</em> لا بد أن يحتوي على
<span dir="ltr">1</span>
في البِتّ الأيمن، لأنه الطريق الوحيد للحصول على عدد فردي في المجموع:
</p>

<div dir="ltr" style="unicode-bidi: isolate; text-align: left;">
$$
1 = 1\times 2^0
$$
</div>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>
لذلك، القيم العشرية الزوجية فقط تقابل الحالة التي تكون فيها «عملية استرجاع رطوبة التربة ذات جودة موصى بها».
</strong>
يمكننا التحقق من ذلك أدناه باستخدام الدالة
<span dir="ltr"><code>np.unpackbits()</code></span>
التي تحول الأعداد العشرية إلى تمثيلات ثنائية.
</p>

</div>


In [ ]:
flags = np.unique(ds['retrieval_qual_flag_dca_pm'].to_numpy())

flags = flags.astype(np.uint8).reshape((flags.size, 1))
flags

In [ ]:
np.unpackbits(flags, axis = 1)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>لماذا يبدو هذا معقّدًا إلى هذا الحد؟</strong>
إن «حزم» أعلام
<span dir="ltr">QA</span>
على مستوى البِتّ هو إرث من الأيام التي كانت فيها مساحة التخزين مكلفة،
وكانت الملفات بحاجة لأن تكون صغيرة قدر الإمكان.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
اليوم، في مجموعات البيانات الهرمية مثل ملفات
<span dir="ltr">HDF5</span>،
يمكننا الاستفادة من <em>ضغط الملفات</em> لتقليل الحجم.
لكن بعض العادات يصعب تغييرها،
ولا تزال العديد من مجموعات بيانات ناسا تستخدم حزم البِتّ
لتخزين معلومات
<span dir="ltr">QA</span>.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يخبرنا عامل باقي القسمة
<span dir="ltr"><code>%</code></span>
إن كان العدد زوجيًا أم فرديًا، لأنه يعطينا <em>باقي</em> القسمة الصحيحة.
</p>

</div>


In [ ]:
# Which integers are evenly divisible by 2?
np.array([3, 8, 16]) % 2

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
دعونا نُحدّث دالة
<span dir="ltr"><code>process_smap_l3()</code></span>
مرةً أخرى.
</p>

</div>


In [ ]:
def process_smap_l3(file_path, threshold = 0.2):
    '''
    Parameters
    ----------
    file_path : str
        The file path to the SMAP L3 file
    threshold : float
        The fractional water area threshold to use when masking out
        permnanet water bodies

    Returns
    -------
    xarray.Dataset
    '''
    with h5py.File(file_path, 'r') as hdf:
        ds = xr.open_dataset(file_path, group = 'Soil_Moisture_Retrieval_Data_PM')
        ds = ds.assign_coords({'x': hdf['x'][:], 'y': hdf['y'][:]})

    # Create a binary (0 or 1) array based on the water fraction threshold
    water_mask = np.where(ds['static_water_body_fraction_pm'][:] >= threshold, 1, 0)
    # Create a binary array based on the QA information
    qa_mask = np.where(ds['retrieval_qual_flag_dca_pm'][:] % 2 != 0, 1, 0)
    
    # Read the dataset out as a NumPy array so we can mask it
    data = ds['soil_moisture_dca_pm'].to_numpy()
    
    # Write NaN into this dataset wherever there are permanent water bodies
    data[water_mask == 1] = np.nan
    # And also mask out low-quality values
    data[qa_mask == 1] = np.nan
    ds['soil_moisture_dca_pm'][:] = data
    return ds

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<hr />

<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
إنشاء سلسلة زمنية لرطوبة التربة
</h2>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
سيكون من الرائع لو استطعنا استخدام
<span dir="ltr"><code>xr.open_mfdataset()</code></span>
لفتح جميع ملفات
<span dir="ltr">SMAP</span>
من نوع
<span dir="ltr">HDF5</span>
كسلسلة زمنية واحدة.
لكن عند المحاولة، سنجد أن ذلك لا يعمل،
لأن
<span dir="ltr">xarray</span>
لا يعرف ما هي إحداثيات مجموعات بيانات
<span dir="ltr">HDF5</span>،
وبالتالي لا يستطيع دمجها تلقائيًا.
</p>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
سيكون من الجميل لو استطعنا استخدام
<span dir="ltr"><code>xr.open_mfdataset()</code></span>
لفتح جميع ملفات
<span dir="ltr">SMAP</span>
من نوع
<span dir="ltr">HDF5</span>
كسلسلة زمنية واحدة.
لكن إذا حاولنا ذلك، سنجد أنه لا يعمل، لأن
<span dir="ltr">xarray</span>
لا يعرف ما هي إحداثيات مجموعات بيانات
<span dir="ltr">HDF5</span>،
وبالتالي لا يستطيع ترتيب مجموعات البيانات ودمجها معًا.
</p>

</div>


In [ ]:
ds = xr.open_mfdataset(
    'data_raw/SMAP_L3/*.h5',
    group = 'Soil_Moisture_Retrieval_Data_PM'
)


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>
هذا يعني أننا سنحتاج إلى فتح كل ملف بأنفسنا وتكديس المصفوفات معًا.
</strong>
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يمكننا استخدام مكتبة
<span dir="ltr"><code>glob</code></span>
للحصول على قائمة بجميع الملفات التي نريدها.
النمط
<span dir="ltr"><code>'data_raw/SMAP_L3/*.h5'</code></span>
مشابه لما استخدمناه مع
<span dir="ltr"><code>xr.open_mfdataset()</code></span>:
فنحن نطلب جميع ملفات
<span dir="ltr">HDF5</span>
داخل مجلد معيّن.
</p>

</div>


In [ ]:
import glob

file_list = glob.glob('data_raw/SMAP_L3/*.h5')

# Get the first filename
file_list[0]

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

#### 🚩 <span style="color:red">انتبه</span>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>
عند استخدام
<span dir="ltr"><code>glob.glob()</code></span>
مع ملفات تمثّل سلسلة زمنية،
من المهم جدًا التأكد من أن الملفات مرتبة زمنيًا!
</strong>
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
طالما أن أسماء الملفات تتضمن طابعًا زمنيًا منطقيًا،
مثل التاريخ بصيغة
<span dir="ltr"><code>YYYYMMDD</code></span>
(سنة–شهر–يوم)،
فإن استخدام الدالة
<span dir="ltr"><code>sort()</code></span>
يعطينا ترتيبًا أبجديًا–رقميًا
يطابق الترتيب الزمني.
</p>

</div>


In [ ]:
file_list.sort()
file_list[0]

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>
دعونا نكتب حلقة
<span dir="ltr"><code>for</code></span>
لمعالجة كل حبيبة
<span dir="ltr">SMAP L3</span>
واستخراج قيمة رطوبة التربة المقدَّرة
في منطقة اهتمامنا
(<span dir="ltr">105.882° W</span>،
<span dir="ltr">48.059° N</span>).
</strong>
</p>

</div>


In [ ]:
# The row, column coordinates of Glasgow, Montana; these could be
#   derived by using the "pyproj" library and an affine transformation
row, column = (51, 198)

sm_mean = []
for filename in file_list:
    ds = process_smap_l3(filename, threshold = 0.2)
    sm_mean.append(ds['soil_moisture_dca_pm'][row,column])

sm_mean = np.hstack(sm_mean)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
عند رسم البيانات، سيكون من المفيد عرض التواريخ على المحور الأفقي.
يمكننا إنشاء تسلسل زمني باستخدام الدالة
<span dir="ltr"><code>pandas.date_range()</code></span>.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
كما سنرغب في ملء القيم المفقودة في سلسلتنا الزمنية لتحسين العرض.
يُعد «الملء للأمام»
(<span dir="ltr">forward filling</span>)
أسلوبًا بسيطًا:
عندما نواجه قيمة مفقودة، نكرر آخر قيمة غير مفقودة حتى نصل إلى قيمة جديدة.
تجعل الدالتان
<span dir="ltr"><code>ffill()</code></span>
و
<span dir="ltr"><code>bfill()</code></span>
في
<span dir="ltr">pandas</span>
هذا الأمر سهل التنفيذ.
</p>

</div>


In [ ]:
import pandas

dates = pandas.date_range('2017-06-01', '2017-09-30', freq = '1D')
df = pandas.DataFrame({'date': dates, 'SM': sm_mean})

# Forward-filling of missing soil moisture (SM) values, followed by backward-filling
df['SM_filled'] = df['SM'].ffill().bfill()
df

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
وأخيرًا، أصبحنا جاهزين لرسم السلسلة الزمنية!
</p>

</div>


In [ ]:
pyplot.figure(figsize = (10, 5))
pyplot.plot(df['date'], df['SM_filled'], 'k-')
pyplot.ylabel('Volumetric Soil Moisture (m3 m-3)')
pyplot.show()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
قد تبدو القفزات غريبة في البداية، لكنها تشبه كثيرًا ما نتوقع رؤيته بعد أحداث هطول مطري
تليها فترات جفاف سريعة.
يمكننا التحقق من أن هذه القفزات تقابل أحداثًا مطرية عبر رسم
<a dir="ltr" href="https://www.weather.gov/wrh/Climate?wfo=ggw">أحداث هطول معروفة قرب <span dir="ltr">Wolf Point</span>، مونتانا</a>
(أقرب محطة طقس إلى منطقة اهتمامنا).
</p>

</div>


In [ ]:
import datetime

precip_events = [
    datetime.date(2017, 6, 12),
    datetime.date(2017, 7, 11),
    datetime.date(2017, 8, 2),
    datetime.date(2017, 9, 15)
]

pyplot.figure(figsize = (10, 5))
pyplot.vlines(precip_events, ymin = df['SM_filled'].min(), ymax = df['SM_filled'].max())
pyplot.plot(df['date'], df['SM_filled'], 'k-', alpha = 0.6)
pyplot.ylabel('Volumetric Soil Moisture (m3 m-3)')
pyplot.show()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بعض القفزات لا ترتبط بأحداث مطر معروفة في هذه المحطة،
لكن ينبغي أن نتذكر أن هذه المحطة تمثل جزءًا صغيرًا من بكسل بحجم
<span dir="ltr">36</span>
كم؛
ومن الممكن أن المطر هطل في مكان ما داخل مربع
<span dir="ltr">36</span>
كم، لكن ليس في موقع محطة الطقس نفسها.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
الفترة الزمنية المرتبطة بجفاف الومضة (بداية سبتمبر) لا تبدو جافة جدًا مقارنة بالتواريخ الأقدم،
رغم أن قمم رطوبة التربة أقل.
مرة أخرى، قد لا تكون هذه المحطة ممثلة تمامًا لظروف إقليمية.
</p>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<hr />

<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
الملخص
</h2>

<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
  <li>
    يوفر
    <span dir="ltr">NASA Earthdata Search</span>
    العديد من مجموعات البيانات لدراسة نظام مناخ الأرض.
    تدمج مجموعات بيانات إعادة التحليل وبيانات ناسا المستوى
    <span dir="ltr">4</span>
    مصادر متعددة لتوفير سجل متصل وتغطية مكانية شبه كاملة، لكنها قد تتضمن انحيازات نمذجة لا تعكس ظروف العالم الحقيقي.
    بينما توفر بيانات الاستشعار عن بعد وبيانات ناسا المستوى
    <span dir="ltr">3</span>
    ملاحظات أكثر مباشرة.
  </li>
  <li>
    <strong>وثّق بياناتك!</strong>
    عند تنزيل البيانات الخام، احتفظ بها منفصلة وغير معدلة.
    تأكد من إنشاء ملف
    <span dir="ltr"><code>README</code></span>
    في المجلد نفسه أو في المجلد الأب، يتضمن معلومات عن مصدر البيانات وكيف ينبغي استخدامها.
  </li>
  <li>
    <strong>وثّق عمليتك!</strong>
    عند العمل مع البيانات، يمكن لدوال بايثون القابلة لإعادة الاستخدام تسريع عملك.
    كما تعمل الدوال المكتوبة جيدًا كتوثيق لسير عملك لأنها تصف الخطوات التي اتبعتها لمعالجة البيانات.
  </li>
  <li>
    <strong>
      يمكن لملفات
      <span dir="ltr">HDF5</span>
      و
      <span dir="ltr">netCDF4</span>
      مساعدتك على البقاء منظمًا.
    </strong>
    يتيح كلا التنسيقين إضافة <em>سمات</em> إلى مجموعات البيانات، وهي طريقة جيدة لتوثيق وحدات القياس ومصادر البيانات الأصلية وغيرها من البيانات الوصفية الأساسية.
  </li>
</ul>

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
قراءة ملفات <span dir="ltr">HDF5</span> و<span dir="ltr">netCDF4</span>
</h3>

</div>


|                            |  HDF5 files                        | netCDF4 files                          | `xarray` (for both)        |
|:---------------------------|:-----------------------------------|:---------------------------------------|:---------------------------|
|Module import               | `import h5py`                      | `import netCDF4`                       | `import xarray as xr`      |
|Files opened with...        | `hdf = h5py.File(...)`             | `nc = netCDF4.Dataset()`               | `ds = xr.open_dataset()`   |
|Datasets/groups viewed...   | `hdf.keys()`                       | `nc.variables` or `nc.variables.keys()`| `list(ds.variables.keys())`|
|                            | `hdf['group_name'].keys()`         | `nc.variables['group_name'].keys()`    |                            |
|Datasets accessed through...| `hdf`                              | `nc.variables`                         | `ds.variables`             |
|Attributes listed through...| `hdf.attrs`                        | `nc.ncattrs()`                         | `ds.attrs`                 |
|                            | `hdf['dataset'].attrs`             | `nc.variables['dataset'].ncattrs()`    |                            |
|Attributes read by...       | `hdf['dataset'].attrs['attribute']`| `nc.variables['dataset'].getncattr()`  | `ds.variables['dataset']`  |

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<hr />

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
موارد إضافية
</h3>

<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
  <li>
    <a dir="ltr" href="https://nsidc.org/data/user-resources/help-center/guide-ease-grids#anchor-9-km-resolution-ease-grids">
      اقرأ المزيد عن شبكات <span dir="ltr">EASE</span> هنا.
    </a>
  </li>
  <li>
    هل أنت فضولي حول كيفية استخدام
    <span dir="ltr"><code>earthaccess.open()</code></span>
    مع
    <span dir="ltr"><code>xarray</code></span>
    بحيث لا تحتاج إلى الاحتفاظ بأي ملفات تم تنزيلها؟
    قد تكون
    <span dir="ltr"><code>xarray.open_dataset()</code></span>
    بطيئة عندما يكون لديك الكثير من الملفات لفتحها، كما في مثال السلسلة الزمنية هذا.
    يصف المقال المرتبط كيف يمكنك تسريع
    <span dir="ltr"><code>xarray.open_dataset()</code></span>
    عند العمل مع عدة ملفات مستضافة على السحابة.
  </li>
</ul>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
المراجع
</h3>

<div dir="ltr" style="unicode-bidi: isolate; text-align: left;">

<ul>
  <li>
    Chen, L. G., J. Gottschalck, A. Hartman, D. Miskus, R. Tinker, and A. Artusa. 2019.
    <em>Flash drought characteristics based on U.S. Drought Monitor.</em>
    Atmosphere 10 (9): 498.
  </li>
  <li>
    He, M., J. S. Kimball, Y. Yi, S. W. Running, K. Guan, K. Jensco, B. Maxwell, and M. Maneta. 2019.
    <em>Impacts of the 2017 flash drought in the US Northern Plains informed by satellite-based evapotranspiration and solar-induced fluorescence.</em>
    Environmental Research Letters 14 (7): 074019.
  </li>
</ul>

</div>
</div>
